In [19]:
# check if display name validation works

import os
import sys

os.environ["LCD_COLUMNS"] = "16"

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)



In [ ]:
baro_result = {
    "temperature": {
        "value": 34.9,
        "unit": "°C",
        "identifier": "temperature_up",
        "display_name": "temperature up",
    },
    "humidity": {
        "value": 87,
        "unit": "%",
        "identifier": "humidity_up",
        "display_name": "humidity up",
    },
    "air_pressure": {
        "value": 10000,
        "unit": "hPa",
        "identifier": "air_pressure_up",
        "display_name": "air pressure up",
    },
}

type(baro_result)

dict

In [16]:
result_dict = {}
display_dict = {}

for _, baro_measure in baro_result.items():
    identifier = baro_measure['identifier']
    baro_value = baro_measure['value']
    result_dict[identifier] = baro_value
    display_dict[baro_measure['display_name']] = f"{baro_value} {baro_measure['unit']}"

print(result_dict)
print(display_dict)

{'temperature_up': 34.9, 'humidity_up': 87, 'air_pressure_up': 10000}
{'temperature up': '34.9 °C', 'humidity up': '87 %', 'air pressure up': '10000 hPa'}


In [1]:
"Up".lower()

'up'

# InfluxDB queries

In [7]:
from datetime import datetime, timedelta, timezone
from dotenv import load_dotenv

load_dotenv()

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.shared.influxdb import setup_client

client = await setup_client()

start_time = datetime.now(tz=timezone.utc) - timedelta(hours=1)
end_time = datetime.now(tz=timezone.utc)

measurement = None
field_identifier = None

#measurement = "Temperature"
#field_identifier = ["temperature_inside", "temperature_outside"]

query = f"""
from(bucket: "greenhouse")
    |> range(start: {start_time.isoformat()}, stop: {end_time.isoformat()})
"""

if measurement:
    query += f"""    |> filter(fn: (r) => r["_measurement"] == "{measurement}")"""

if field_identifier:
    filters = ' or '.join([f'r["_field"] == "{ident}"' for ident in field_identifier])
    query += f"""    |> filter(fn: (r) => {filters})"""

async with client:
    db_query = " ".join(line.strip() for line in query.splitlines())
    table = await client.query_api().query(
        query=db_query
    )

columns=["_measurement", "_field", "_time", "_value"]

result = table.to_values(columns=columns)

In [8]:
result

[['AirPressure',
  'air_pressure',
  datetime.datetime(2025, 12, 4, 11, 32, 23, 684338, tzinfo=datetime.timezone.utc),
  1011.53],
 ['AirPressure',
  'air_pressure',
  datetime.datetime(2025, 12, 4, 11, 33, 26, 484129, tzinfo=datetime.timezone.utc),
  1011.63],
 ['AirPressure',
  'air_pressure',
  datetime.datetime(2025, 12, 4, 11, 34, 29, 316301, tzinfo=datetime.timezone.utc),
  1011.62],
 ['AirPressure',
  'air_pressure',
  datetime.datetime(2025, 12, 4, 11, 35, 32, 146061, tzinfo=datetime.timezone.utc),
  1011.59],
 ['AirPressure',
  'air_pressure',
  datetime.datetime(2025, 12, 4, 11, 36, 34, 985808, tzinfo=datetime.timezone.utc),
  1011.52],
 ['AirPressure',
  'air_pressure',
  datetime.datetime(2025, 12, 4, 11, 37, 37, 696751, tzinfo=datetime.timezone.utc),
  1011.47],
 ['AirPressure',
  'air_pressure',
  datetime.datetime(2025, 12, 4, 11, 38, 40, 367183, tzinfo=datetime.timezone.utc),
  1011.45],
 ['AirPressure',
  'air_pressure',
  datetime.datetime(2025, 12, 4, 11, 39, 43, 176